# Machine Learning Option Pricing Pipeline

This notebook demonstrates how to generate synthetic European call option data using the Black–Scholes formula, train several machine‑learning models to approximate option prices, and discuss how sequence models (LSTM/GRU) could be applied when sequential features are available.


## Theoretical background

Under the Black–Scholes model, the underlying stock price $S_t$ follows geometric Brownian motion: $$\mathrm{d}S_t = \mu S_t\,\mathrm{d}t + \sigma S_t\,\mathrm{d}W_t$$, which has the solution $S_t = S_0 \exp[(\mu - \sigma^2/2)t + \sigma W_t]$.  The price of a European call option with strike $K$, maturity $t$ and risk‑free rate $r$ is given by

$$C = S N(d_1) - K e^{-r t} N(d_2),$$

where $d_1 = rac{\ln(S/K) + (r + \sigma^2/2)t}{\sigma\sqrt{t}}$ and $d_2 = d_1 - \sigma \sqrt{t}$.


In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt

# Fix random seed for reproducibility
np.random.seed(42)

# Black–Scholes call option price function
def black_scholes_call_price(S, K, t, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * t) / (sigma * np.sqrt(t))
    d2 = d1 - sigma * np.sqrt(t)
    return S * norm.cdf(d1) - K * np.exp(-r * t) * norm.cdf(d2)

In [ ]:
# Generate synthetic data
n_samples = 5000

S = np.random.uniform(20, 150, n_samples)      # current stock prices
K = np.random.uniform(10, 200, n_samples)      # strike prices
t = np.random.uniform(0.1, 2.0, n_samples)     # time to maturity in years
sigma = np.random.uniform(0.1, 0.6, n_samples) # volatility (10% to 60%)
r = np.random.uniform(0.01, 0.05, n_samples)   # risk‑free rate (1% to 5%)

prices = black_scholes_call_price(S, K, t, r, sigma)

# Assemble data frame
data = pd.DataFrame({
    'S': S,
    'K': K,
    't': t,
    'sigma': sigma,
    'r': r,
    'price': prices
})

# Split into training and test sets
X = data[['S', 'K', 't', 'sigma', 'r']]
y = data['price']
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Train Linear Regression model
lin_reg = LinearRegression()
lin_reg.fit(train_X, train_y)
pred_lin = lin_reg.predict(test_X)

# Train Random Forest Regressor
rf_reg = RandomForestRegressor(n_estimators=200, random_state=42)
rf_reg.fit(train_X, train_y)
pred_rf = rf_reg.predict(test_X)

# Train MLP Regressor (neural network)
mlp_reg = MLPRegressor(hidden_layer_sizes=(100,), activation='relu', solver='adam', random_state=42, max_iter=500)
mlp_reg.fit(train_X, train_y)
pred_mlp = mlp_reg.predict(test_X)

# Evaluate models
models = {'Linear Regression': pred_lin, 'Random Forest': pred_rf, 'MLP Regressor': pred_mlp}
for name, pred in models.items():
    rmse = mean_squared_error(test_y, pred, squared=False)
    mae = mean_absolute_error(test_y, pred)
    r2 = r2_score(test_y, pred)
    print(f"{name}: RMSE={rmse:.4f}, MAE={mae:.4f}, R2={r2:.4f}")

In [ ]:
# Plot predictions vs true prices for a sample of 200 points
sample_idx = np.random.choice(len(test_y), 200, replace=False)
true_sample = test_y.iloc[sample_idx].values
plt.figure(figsize=(10,5))
for name, pred in models.items():
    pred_sample = pred[sample_idx]
    plt.scatter(true_sample, pred_sample, label=name, alpha=0.7)
plt.xlabel('True Price')
plt.ylabel('Predicted Price')
plt.title('Predicted vs True Option Prices (sample 200)')
plt.legend()
plt.grid(False)
plt.show()

In [ ]:
# Simulate GBM paths to visualise underlying dynamics
n_paths = 10
n_steps = 252  # daily steps for one year
delta_t = 1/252
time = np.linspace(0, 1, n_steps)
mu = 0.05
vol = 0.2
S0 = 100

gbm_paths = np.zeros((n_paths, n_steps))
gbm_paths[:,0] = S0
for i in range(1, n_steps):
    z = np.random.normal(size=n_paths)
    gbm_paths[:, i] = gbm_paths[:, i-1] * np.exp((mu - 0.5 * vol**2) * delta_t + vol * np.sqrt(delta_t) * z)

plt.figure(figsize=(8,5))
for j in range(n_paths):
    plt.plot(time, gbm_paths[j], linewidth=1)
plt.xlabel('Time (years)')
plt.ylabel('Stock Price')
plt.title('Sample Geometric Brownian Motion Paths')
plt.show()

## LSTM/GRU skeleton code

The following illustrates how to build an LSTM‑based regressor using PyTorch.  It accepts an input tensor of shape `(batch_size, sequence_length, n_features)` and outputs a single price prediction per sequence:

    import torch
    import torch.nn as nn
    from torch.utils.data import DataLoader, TensorDataset

    class LSTMRegressor(nn.Module):
        def __init__(self, input_dim: int, hidden_dim: int = 64, num_layers: int = 2):
            super().__init__()
            self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
            self.fc = nn.Linear(hidden_dim, 1)

        def forward(self, x):
            # x: [batch_size, seq_len, input_dim]
            out, _ = self.lstm(x)
            out = out[:, -1, :]
            return self.fc(out).squeeze(1)

    # Example usage (assumes train_X is a tensor of shape [n_samples, seq_len, n_features])
    model = LSTMRegressor(input_dim=train_X.size(2))
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    train_dataset = TensorDataset(train_X, train_y)
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

    for epoch in range(50):
        model.train()
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            pred = model(X_batch)
            loss = criterion(pred, y_batch)
            loss.backward()
            optimizer.step()

To construct a GRU‑based model simply replace `nn.LSTM` with `nn.GRU`.  In practice, one would use actual historical price sequences and other time‑varying features as inputs.
